# Image Classification: Dogs and Cats
Trains a model to classify an image as a cat or a dog using 25,000 images.


Below we will do the following:

1. Setup the training environment.
2. Load 25,000 cat and dog images.
3. Build a classification model to predict whether an image is a cat or a dog.
3. Convert the model to CoreML and upload it to Skafos.

The example is based on [Turi Create's Image Classifier](https://apple.github.io/turicreate/docs/userguide/image_classifier/).

## Environment Setup
Below we ensure `CUDA 10` is installed and then use pip to install `turicreate`, `mxnet-cu100`, and `skafos` libraries.

In [0]:
# Confirm that you have CUDA 10
!nvcc --version

In [0]:
# Install libraries - you might need to restart the runtime after doing this
!pip install turicreate==5.4
# The wrong version of mxnet will be installed
!pip uninstall -y mxnet
# Install CUDA10-compatible version of mxnet
!pip install mxnet-cu100
# install Skafos python sdk
!pip install skafos

## Data Preparation and Model Training
The training data for this example are 25,000 images, 12,500 cats and 12,500 dogs. The original data set is [here](https://www.microsoft.com/en-us/download/details.aspx?id=54765) and we have also included it in the public bucket listed below.

After unzipping and extracting the images, they are loaded into a Turi Create SFrame and labels are created for each image based on the path. The data is randomly split into train and test sets, where 80% of the data is used for training and 20% is used for model evaluation (if you desire). Training this model with a GPU is much faster than CPU time. By default, this runtime environment should be using a Python 3 GPU backend instance. Below, we tell Turicreate to use all available GPUs for processing.

In [0]:
# Import libraries and tell Turicreate to use all GPUs available - this may throw a warning
import urllib
import tarfile

import coremltools
import turicreate as tc
tc.config.set_num_gpus(-1)

In [0]:
# Specify the data set download url
data_url = "https://s3.amazonaws.com/skafos.example.data/ImageClassifier/PetImages.tar.gz"
data_path = "PetImages.tar.gz"

# Pull the compressed data and extract it
retrieve = urllib.request.urlretrieve(data_url, data_path)
tar = tarfile.open(data_path)
tar.extractall()
tar.close()

In [0]:
# Load images (Note: you can ignore 'Unexpected JPEG decode failure' errors)
data = tc.image_analysis.load_images('PetImages', with_path=True, ignore_failure=True)

# From the path-name, create a label column. This labels each image as either a dog or a cat 
data['label'] = data['path'].apply(lambda path: 'dog' if '/Dog' in path else 'cat')

# Make a train-set split
train_data, test_data = data.random_split(0.8)

In [0]:
train_data.head()

In [0]:
# Train an image classification model - consider increasing max_iterations
model = tc.image_classifier.create(
    dataset=train_data,
    target='label',
    model='resnet-50',
    batch_size=32,
    max_iterations=30
)

# Image Classification Training Docs:
# https://apple.github.io/turicreate/docs/api/generated/turicreate.image_classifier.create.html#turicreate.image_classifier.create

## Model Export and Skafos Upload
- Convert the model to CoreML format so that it can run on an iOS device. Then deliver the model to your apps with **[Skafos](https://skafos.ai)**.

- If you don't already have an account, Sign Up for one **[here](https://dashboard.skafos.ai)**. 
- Once you've signed up for an account, grab an API token from your account settings.

In [0]:
# Specify the CoreML model name
model_name = 'ImageClassifier'
coreml_model_name = model_name + '.mlmodel'

# Export the trained model to CoreML format
res = model.export_coreml(coreml_model_name) 


In [0]:
import skafos
from skafos import models
import os

# Set your API Token first for repeated use
os.environ["SKAFOS_API_TOKEN"] = "<YOUR-SKAFOS-API-TOKEN>"

# You can retrieve this info with skafos.summary()
org_name = "<YOUR-SKAFOS-ORG-NAME>"    # Example: "mike-gmail-com-467h2"
app_name = "<YOUR-SKAFOS-APP-NAME>"    # Example: "ImageClassification-App"
model_name = "<YOUR-MODEL-NAME>"       # Example: "ImageClassificationModel"

# Upload model version to Skafos
model_upload_result = models.upload_version(
    files="ImageClassifier.mlmodel",
    org_name=org_name,
    app_name=app_name,
    model_name=model_name
)